In [16]:
pip install streamlit

In [17]:
import streamlit as st
import torch
import torchvision.transforms as transforms
from torchvision.datasets import MNIST
import matplotlib.pyplot as plt
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegressionCV
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

In [18]:
mnist_transform = transforms.Compose([transforms.ToTensor()])
mnist_train = MNIST(root='data', train=True, transform=mnist_transform, download=True)
mnist_test = MNIST(root='data', train=False, transform=mnist_transform, download=True)

In [19]:
mnist = fetch_openml('mnist_784')
X, y = mnist.data.to_numpy(), mnist.target.to_numpy()
X = X / 255.0
y = y.astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

/usr/local/lib/python3.10/dist-packages/sklearn/datasets/_openml.py:1022: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [20]:
# Train the Random Forest model
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [22]:
# Train the Logistic Regression model with cross-validation
log_reg_model = LogisticRegressionCV(max_iter=10, cv=5, random_state=42)
log_reg_model.fit(X_train, y_train)
cv_scores = log_reg_model.scores_[1].mean(axis=0)
st.write(f"Logistic RegressionCV Best Score: {cv_scores.max():.4f}")

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _c

In [23]:
def predict_image(img, model):
    img = img.reshape(1, -1)
    return model.predict(img)[0]

In [24]:
# Streamlit app
st.title("MNIST Digit Detection")

img_index = st.number_input("Enter an image index (0-9999):", min_value=0, max_value=9999, value=0)

if st.button("Predict"):
    if img_index < len(X_test):
        img = X_test[img_index]
        label = y_test[img_index]

        # Predict using Random Forest
        rf_predicted_label = predict_image(img, rf_model)

        # Predict using Logistic RegressionCV
        log_reg_predicted_label = predict_image(img, log_reg_model)

        st.write(f"Label: {label}, RF Predicted: {rf_predicted_label}, Log Reg Predicted: {log_reg_predicted_label}")

        fig, ax = plt.subplots()
        ax.imshow(img.reshape(28, 28), cmap='gray')
        ax.set_title(f"Label: {label}, RF Predicted: {rf_predicted_label}, Log Reg Predicted: {log_reg_predicted_label}")
        st.pyplot(fig)
    else:
        st.write(f"Index {img_index} is out of bounds for the test dataset.")